# Compare Two WandB Runs Side by Side

This notebook fetches and visualizes data from two experiment runs for easy comparison.

In [ ]:
# Import Required Libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any, Optional, List
import json
from collections import defaultdict

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, HTML, Markdown

# Import wandb
import wandb
from wandb.apis.public import Run

# Set style for plots
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")

In [ ]:
# Initialize Experiment Tracking - Define Run IDs
RUN_IDS = ['poxmea6n', 'buqt4b6u']
ENTITY = 'mllab-ts-universit-di-trieste'
PROJECT = 'CounterFactualDPG'

print(f"Comparing {len(RUN_IDS)} runs:")
for i, run_id in enumerate(RUN_IDS, 1):
    print(f"  {i}. {run_id}")
print(f"\nEntity: {ENTITY}")
print(f"Project: {PROJECT}")

In [ ]:
# Load Run Data from WandB
api = wandb.Api()

def fetch_run_data(run_id: str) -> Dict[str, Any]:
    """Fetch comprehensive information from a WandB run."""
    run_path = f"{ENTITY}/{PROJECT}/{run_id}"
    print(f"Fetching run: {run_path}")
    
    try:
        run = api.run(run_path)
        
        # Collect all run information
        run_data = {
            'meta': {
                'id': run.id,
                'name': run.name,
                'display_name': run.display_name,
                'state': run.state,
                'url': run.url,
                'path': run.path,
                'entity': run.entity,
                'project': run.project,
                'created_at': run.created_at,
                'updated_at': getattr(run, 'updated_at', None),
                'notes': run.notes,
                'tags': list(run.tags) if run.tags else [],
                'group': run.group,
                'job_type': run.job_type,
            },
            'config': dict(run.config),
            'summary': {},
            'history': [],
            'history_keys': [],
            'system_metrics': {},
            'files': [],
            'artifacts': [],
        }
        
        # Get summary metrics
        for key, value in run.summary.items():
            if not key.startswith('_'):
                try:
                    run_data['summary'][key] = float(value)
                except (ValueError, TypeError):
                    run_data['summary'][key] = value
        
        # Get history (time-series data)
        try:
            history = run.history(pandas=False)
            if history:
                run_data['history'] = list(history)
                # Extract unique keys from history
                all_keys = set()
                for row in history:
                    all_keys.update(row.keys())
                run_data['history_keys'] = sorted(list(all_keys))
        except Exception as e:
            print(f"  Warning: Could not fetch history: {e}")
        
        # Get files
        try:
            files = run.files()
            run_data['files'] = [
                {
                    'name': f.name,
                    'size': f.size,
                    'mimetype': getattr(f, 'mimetype', None),
                    'url': f.url,
                }
                for f in files
            ]
        except Exception as e:
            print(f"  Warning: Could not fetch files: {e}")
        
        # Get artifacts
        try:
            artifacts = run.logged_artifacts()
            run_data['artifacts'] = [
                {
                    'name': a.name,
                    'type': a.type,
                    'version': a.version,
                    'size': a.size,
                }
                for a in artifacts
            ]
        except Exception as e:
            print(f"  Warning: Could not fetch artifacts: {e}")
        
        print(f"  ✓ Successfully fetched {len(run_data['history'])} history steps")
        return run_data
        
    except Exception as e:
        print(f"  ✗ Error fetching run: {e}")
        return None

# Fetch data for all runs
runs_data = {}
for run_id in RUN_IDS:
    runs_data[run_id] = fetch_run_data(run_id)

print(f"\n✓ Loaded data for {len([r for r in runs_data.values() if r])}/{len(RUN_IDS)} runs")

In [ ]:
# Extract Metrics and Parameters for Comparison
def format_value(value, max_length=50):
    """Format values for display."""
    if value is None:
        return "N/A"
    if isinstance(value, (list, dict)):
        return f"{type(value).__name__}({len(value)})"
    str_val = str(value)
    if len(str_val) > max_length:
        return str_val[:max_length-3] + "..."
    return str_val

# Create comparison DataFrames
comparison_data = {
    'Metric': [f"Run {i+1} ({run_id})" for i, run_id in enumerate(RUN_IDS)]
}
for run_id in RUN_IDS:
    comparison_data[f"Run {RUN_IDS.index(run_id)+1} ({run_id})"] = []

# Metadata comparison
meta_fields = ['name', 'state', 'created_at', 'tags', 'group', 'job_type']
for field in meta_fields:
    comparison_data['Metric'].append(f"meta.{field}")
    for run_id in RUN_IDS:
        comparison_data[f"Run {RUN_IDS.index(run_id)+1} ({run_id})"].append(format_value(runs_data[run_id]['meta'].get(field)))

# Summary metrics - find common keys
summary_keys_1 = set(runs_data['poxmea6n']['summary'].keys())
summary_keys_2 = set(runs_data['buqt4b6u']['summary'].keys())
common_summary_keys = sorted(summary_keys_1 & summary_keys_2)

for key in common_summary_keys:
    comparison_data['Metric'].append(f"summary.{key}")
    comparison_data['Run 1 (poxmea6n)'].append(format_value(runs_data['poxmea6n']['summary'].get(key)))
    comparison_data['Run 2 (buqt4b6u)'].append(format_value(runs_data['buqt4b6u']['summary'].get(key)))

# Configuration comparison
config_keys_1 = set(runs_data['poxmea6n']['config'].keys())
config_keys_2 = set(runs_data['buqt4b6u']['config'].keys())
common_config_keys = sorted(config_keys_1 & config_keys_2)

for key in common_config_keys:
    comparison_data['Metric'].append(f"config.{key}")
    comparison_data['Run 1 (poxmea6n)'].append(format_value(runs_data['poxmea6n']['config'].get(key)))
    comparison_data['Run 2 (buqt4b6u)'].append(format_value(runs_data['buqt4b6u']['config'].get(key)))

df_comparison = pd.DataFrame(comparison_data)
print("✓ Extracted metrics and parameters for comparison")
display(df_comparison.head(20))

In [ ]:
# Create Side-by-Side Comparison Tables

# 1. Metadata Comparison
meta_data = []
for run_id in RUN_IDS:
    meta_data.append(runs_data[run_id]['meta'])

df_meta = pd.DataFrame(meta_data, index=['Run 1 (poxmea6n)', 'Run 2 (buqt4b6u)']).T

# 2. Summary Metrics Comparison
df_summary = pd.DataFrame({
    'Run 1 (poxmea6n)': runs_data['poxmea6n']['summary'],
    'Run 2 (buqt4b6u)': runs_data['buqt4b6u']['summary']
})

# 3. Configuration Comparison
df_config = pd.DataFrame({
    'Run 1 (poxmea6n)': runs_data['poxmea6n']['config'],
    'Run 2 (buqt4b6u)': runs_data['buqt4b6u']['config']
})

# Display side by side
HTML("""
<div style="display: flex; gap: 20px; overflow-x: auto;">
    <div style="flex: 1;">
        <h3 style="color: #2e86de; margin-bottom: 10px;">📋 Run Metadata</h3>
        {meta_html}
    </div>
    <div style="flex: 1;">
        <h3 style="color: #10ac84; margin-bottom: 10px;">📊 Summary Metrics</h3>
        {summary_html}
    </div>
</div>
""".format(
    meta_html=df_meta.to_html(classes="data-table"),
    summary_html=df_summary.head(20).to_html(classes="data-table")
))

In [ ]:
# Visualize Training History - Side by Side

# Find common history keys
history_keys_1 = set(runs_data['poxmea6n']['history_keys'])
history_keys_2 = set(runs_data['buqt4b6u']['history_keys'])
common_metrics = sorted(history_keys_1 & history_keys_2)

print(f"Common metrics found in history: {len(common_metrics)}")
print(f"Metrics: {common_metrics}")

# Filter to numeric metrics
numeric_metrics = []
for key in common_metrics:
    # Check if this is a numeric metric
    is_numeric = True
    for run_id in RUN_IDS:
        for row in runs_data[run_id]['history']:
            if key in row:
                val = row[key]
                if val is not None and not isinstance(val, (int, float)):
                    is_numeric = False
                    break
    if is_numeric and any(key in row for row in runs_data['poxmea6n']['history']):
        numeric_metrics.append(key)

print(f"\nNumeric metrics to plot: {numeric_metrics}")

# Create subplots for each metric
if numeric_metrics:
    n_metrics = min(len(numeric_metrics), 8)  # Limit to 8 metrics
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        subplot_titles=numeric_metrics[:n_metrics]
    )
    
    colors = ['#2e86de', '#10ac84']
    
    for idx, metric in enumerate(numeric_metrics[:n_metrics]):
        row = (idx // n_cols) + 1
        col = (idx % n_cols) + 1
        
        for i, run_id in enumerate(RUN_IDS):
            history = runs_data[run_id]['history']
            steps = []
            values = []
            for row_data in history:
                if metric in row_data:
                    steps.append(row_data.get('_step', len(values)))
                    values.append(row_data[metric])
            
            if steps and values:
                fig.add_trace(
                    go.Scatter(
                        x=steps,
                        y=values,
                        mode='lines+markers',
                        name=f"Run {i+1} ({run_id})",
                        legendgroup=f"group_{i}" if idx == 0 else None,
                        showlegend=(idx == 0),
                        line=dict(color=colors[i], width=2),
                        marker=dict(size=4)
                    ),
                    row=row, col=col
                )
    
    fig.update_layout(
        height=300 * n_rows,
        title_text="<b>Training History: Side-by-Side Comparison</b>",
        hovermode='x unified',
        template='plotly_white'
    )
    
    fig.show()
else:
    print("No numeric metrics found in history to plot.")

In [ ]:
# Compare Model Performance - Summary Metrics Bar Chart

# Extract numeric summary metrics for comparison
metric_values = []
metric_names = []

for key in runs_data['poxmea6n']['summary'].keys():
    if not key.startswith('_') and isinstance(runs_data['poxmea6n']['summary'].get(key), (int, float)):
        try:
            val1 = float(runs_data['poxmea6n']['summary'][key])
            val2 = float(runs_data['buqt4b6u'].get('summary', {}).get(key, 0))
            
            # Filter out extreme values for better visualization
            if abs(val1) < 1e6 and abs(val2) < 1e6:
                metric_names.append(key)
                metric_values.append([val1, val2])
        except (ValueError, TypeError):
            pass

if metric_values:
    df_metrics = pd.DataFrame(metric_values, columns=['Run 1 (poxmea6n)', 'Run 2 (buqt4b6u)'], index=metric_names)
    
    # Bar chart comparison
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='Run 1 (poxmea6n)',
        x=metric_names,
        y=df_metrics['Run 1 (poxmea6n)'],
        marker_color='#2e86de'
    ))
    
    fig.add_trace(go.Bar(
        name='Run 2 (buqt4b6u)',
        x=metric_names,
        y=df_metrics['Run 2 (buqt4b6u)'],
        marker_color='#10ac84'
    ))
    
    fig.update_layout(
        title='<b>Summary Metrics Comparison</b>',
        xaxis_title='Metric',
        yaxis_title='Value',
        barmode='group',
        height=600,
        hovermode='x unified',
        xaxis={'tickangle': -45},
        template='plotly_white'
    )
    
    fig.show()
else:
    print("No numeric summary metrics found for comparison.")

In [ ]:
# Metric Difference Analysis

# Calculate percentage differences
differences = []
for key in runs_data['poxmea6n']['summary'].keys():
    if (not key.startswith('_') and 
        key in runs_data['buqt4b6u']['summary'] and
        isinstance(runs_data['poxmea6n']['summary'].get(key), (int, float)) and
        isinstance(runs_data['buqt4b6u']['summary'].get(key), (int, float))):
        
        try:
            val1 = float(runs_data['poxmea6n']['summary'][key])
            val2 = float(runs_data['buqt4b6u']['summary'][key])
            
            if abs(val1) > 1e-10:  # Avoid division by zero
                pct_diff = ((val2 - val1) / abs(val1)) * 100
                absolute_diff = val2 - val1
                
                # Filter for meaningful differences
                if abs(val1) < 1e6 and abs(val2) < 1e6:
                    differences.append({
                        'Metric': key,
                        'Run 1': val1,
                        'Run 2': val2,
                        'Absolute Diff': absolute_diff,
                        'Percent Diff (%)': pct_diff,
                        'Better Run': 'Run 2' if pct_diff > 0 else 'Run 1'
                    })
        except (ValueError, TypeError):
            pass

if differences:
    df_diff = pd.DataFrame(differences).sort_values('Absolute Diff', key=abs, ascending=False)
    
    print("Top Metric Differences:")
    df_diff_top = df_diff.head(15)
    display(df_diff_top)
    
    # Visualization of differences
    fig = go.Figure()
    
    # Color based on whether Run 2 improved
    colors = ['#10ac84' if diff > 0 else '#ee5253' for diff in df_diff_top['Percent Diff (%)']]
    
    fig.add_trace(go.Bar(
        x=df_diff_top['Metric'],
        y=df_diff_top['Percent Diff (%)'],
        marker_color=colors,
        hovertemplate='%{x}<br>Percent Change: %{y:.2f}%<br>Run 1: %{customdata[0]:.4f}<br>Run 2: %{customdata[1]:.4f}<extra></extra>',
        customdata=zip(df_diff_top['Run 1'], df_diff_top['Run 2'])
    ))
    
    # Add horizontal line at 0
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    fig.update_layout(
        title='<b>Percentage Difference: Run 2 vs Run 1</b>',
        xaxis_title='Metric',
        yaxis_title='Percentage Difference (%)',
        height=600,
        xaxis={'tickangle': -45},
        template='plotly_white',
        annotations=[
            dict(x=0.02, y=0.98, 
                 text='<span style="color:#10ac84">● Run 2 better</span> <br> <span style="color:#ee5253">● Run 1 better</span>',
                 showarrow=False, xref='paper', yref='paper', 
                 xanchor='left', yanchor='top', bgcolor='rgba(255,255,255,0.9)',
                 bordercolor='gray', borderwidth=1)
        ]
    )
    
    fig.show()
else:
    print("No comparable metrics found for difference analysis.")

In [ ]:
# Compare Files and Artifacts

print("📁 Files Comparison")
print("=" * 80)

files_data = {'File Name': [], 'Run 1 Size (KB)': [], 'Run 2 Size (KB)': [], 'Available In': []}

# Get all unique file names
all_files = set()
for run_id in RUN_IDS:
    for f in runs_data[run_id]['files']:
        all_files.add(f['name'])

for filename in sorted(all_files):
    available_in = []
    size1 = "N/A"
    size2 = "N/A"
    
    if filename in [f['name'] for f in runs_data['poxmea6n']['files']]:
        available_in.append('Run 1')
        f = next(f for f in runs_data['poxmea6n']['files'] if f['name'] == filename)
        size1 = f"{f['size'] / 1024:.2f}" if f['size'] else "Unknown"
    
    if filename in [f['name'] for f in runs_data['buqt4b6u']['files']]:
        available_in.append('Run 2')
        f = next(f for f in runs_data['buqt4b6u']['files'] if f['name'] == filename)
        size2 = f"{f['size'] / 1024:.2f}" if f['size'] else "Unknown"
    
    files_data['File Name'].append(filename)
    files_data['Run 1 Size (KB)'].append(size1)
    files_data['Run 2 Size (KB)'].append(size2)
    files_data['Available In'].append(', '.join(available_in))

df_files = pd.DataFrame(files_data)
display(df_files.head(20))

print("\n📦 Artifacts Comparison")
print("=" * 80)

artifacts_data = {'Artifact': [], 'Run 1 Version': [], 'Run 2 Version': [], 'Available In': [], 'Type': []}

# Get all unique artifact names
all_artifacts = set()
for run_id in RUN_IDS:
    for a in runs_data[run_id]['artifacts']:
        all_artifacts.add(a['name'])

for artifact_name in sorted(all_artifacts):
    available_in = []
    version1 = "N/A"
    version2 = "N/A"
    artifact_type = "N/A"
    
    if artifact_name in [a['name'] for a in runs_data['poxmea6n']['artifacts']]:
        available_in.append('Run 1')
        a = next(a for a in runs_data['poxmea6n']['artifacts'] if a['name'] == artifact_name)
        version1 = a['version']
        artifact_type = a['type']
    
    if artifact_name in [a['name'] for a in runs_data['buqt4b6u']['artifacts']]:
        available_in.append('Run 2')
        a = next(a for a in runs_data['buqt4b6u']['artifacts'] if a['name'] == artifact_name)
        version2 = a['version']
        artifact_type = a['type']
    
    artifacts_data['Artifact'].append(artifact_name)
    artifacts_data['Run 1 Version'].append(version1)
    artifacts_data['Run 2 Version'].append(version2)
    artifacts_data['Available In'].append(', '.join(available_in))
    artifacts_data['Type'].append(artifact_type)

df_artifacts = pd.DataFrame(artifacts_data)
display(df_artifacts)

print(f"\nSummary:")
print(f"  Run 1 Files: {len(runs_data['poxmea6n']['files'])}")
print(f"  Run 2 Files: {len(runs_data['buqt4b6u']['files'])}")
print(f"  Run 1 Artifacts: {len(runs_data['poxmea6n']['artifacts'])}")
print(f"  Run 2 Artifacts: {len(runs_data['buqt4b6u']['artifacts'])}")

In [ ]:
# Display Images from Runs Side by Side

import requests
from PIL import Image
from io import BytesIO
import base64
from urllib.parse import urlparse

def display_image_side_by_side(run1_image, run2_image, image_name):
    """Display two images side by side."""
    
    def image_to_html(img, title):
        """Convert PIL image to HTML."""
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        return f"""
        <div style="text-align: center; flex: 1; min-width: 300px; padding: 10px;">
            <h4 style="color: {'#2e86de' if 'Run 1' in title else '#10ac84'}; margin-bottom: 10px;">{title}</h4>
            <img src="data:image/png;base64,{img_str}" style="max-width: 100%; height: auto; border: 2px solid #ddd; border-radius: 5px;">
        </div>
        """
    
    if run1_image and run2_image:
        html = f"""
        <div style="display: flex; gap: 20px; flex-wrap: wrap; margin-bottom: 30px;">
            <div style="width: 100%; text-align: center; padding: 10px; background: #f8f9fa; border-radius: 5px; margin-bottom: 10px;">
                <h3 style="margin: 0;">🖼️ {image_name}</h3>
            </div>
            {image_to_html(run1_image, 'Run 1 (poxmea6n)')}
            {image_to_html(run2_image, 'Run 2 (buqt4b6u)')}
        </div>
        """
        display(HTML(html))
    elif run1_image:
        html = f"""
        <div style="text-align: center; padding: 15px; background: #fff3cd; border-radius: 5px; margin-bottom: 30px;">
            <h3 style="color: #856404; margin: 0;">🖼️ {image_name} - Only in Run 1</h3>
        </div>
        <div style="display: flex; justify-content: center;">
            {image_to_html(run1_image, 'Run 1 (poxmea6n)')}
        </div>
        """
        display(HTML(html))
    elif run2_image:
        html = f"""
        <div style="text-align: center; padding: 15px; background: #fff3cd; border-radius: 5px; margin-bottom: 30px;">
            <h3 style="color: #856404; margin: 0;">🖼️ {image_name} - Only in Run 2</h3>
        </div>
        <div style="display: flex; justify-content: center;">
            {image_to_html(run2_image, 'Run 2 (buqt4b6u)')}
        </div>
        """
        display(HTML(html))
    else:
        print(f"⚠️  No image found for: {image_name}")

def download_image(url):
    """Download image from URL and return PIL Image."""
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            return Image.open(BytesIO(response.content))
    except Exception as e:
        print(f"  Error downloading image from {url}: {e}")
    return None

def get_image_files(run_data):
    """Get all image files from run data."""
    image_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.bmp', '.svg', '.webp'}
    image_files = []
    
    for f in run_data['files']:
        name_lower = f['name'].lower()
        if any(name_lower.endswith(ext) for ext in image_extensions):
            image_files.append(f)
    
    return image_files

print("🖼️ Image Comparison")
print("=" * 80)

# Get image files from both runs
run1_images = get_image_files(runs_data['poxmea6n'])
run2_images = get_image_files(runs_data['buqt4b6u'])

print(f"Run 1: {len(run1_images)} images found")
for img in run1_images:
    size_str = f"{img['size'] / 1024:.2f} KB" if img['size'] else "Unknown"
    print(f"  - {img['name']} ({size_str})")

print(f"\nRun 2: {len(run2_images)} images found")
for img in run2_images:
    size_str = f"{img['size'] / 1024:.2f} KB" if img['size'] else "Unknown"
    print(f"  - {img['name']} ({size_str})")

# Collect all unique image names for comparison
all_image_names = set()
all_image_names.update([img['name'] for img in run1_images])
all_image_names.update([img['name'] for img in run2_images])

print(f"\n{'=' * 80}")
print("Displaying Images Side by Side")
print("=" * 80)

if not all_image_names:
    print("⚠️  No images found in either run.")
else:
    # Display each image comparison
    for img_name in sorted(all_image_names):
        # Get image from Run 1
        run1_img_file = next((f for f in run1_images if f['name'] == img_name), None)
        run1_image = None
        if run1_img_file:
            print(f"Downloading from Run 1: {img_name}")
            run1_image = download_image(run1_img_file['url'])
        
        # Get image from Run 2
        run2_img_file = next((f for f in run2_images if f['name'] == img_name), None)
        run2_image = None
        if run2_img_file:
            print(f"Downloading from Run 2: {img_name}")
            run2_image = download_image(run2_img_file['url'])
        
        # Display side by side
        display_image_side_by_side(run1_image, run2_image, img_name)
        print()

print("✓ Image comparison complete!")

In [ ]:
# Summary and Quick Links

print("=" * 80)
print("📊 RUN COMPARISON SUMMARY")
print("=" * 80)

for i, run_id in enumerate(RUN_IDS, 1):
    meta = runs_data[run_id]['meta']
    summary = runs_data[run_id]['summary']
    
    print(f"\n{'─' * 80}")
    print(f"🏃 RUN {i}: {run_id}")
    print(f"{'─' * 80}")
    print(f"Name:        {meta['name']}")
    print(f"Display:     {meta['display_name']}")
    print(f"State:       {meta['state']}")
    print(f"Created:     {meta['created_at']}")
    print(f"Tags:        {', '.join(meta['tags']) if meta['tags'] else 'None'}")
    print(f"Group:       {meta['group'] or 'None'}")
    print(f"Job Type:    {meta['job_type']}")
    print(f"\n🔗 WandB URL: {meta['url']}")
    print(f"\n📈 History:   {len(runs_data[run_id]['history'])} steps, {len(runs_data[run_id]['history_keys'])} metrics")
    print(f"📁 Files:     {len(runs_data[run_id]['files'])}")
    print(f"📦 Artifacts: {len(runs_data[run_id]['artifacts'])}")

print(f"\n{'=' * 80}")
print("Key Statistics")
print("=" * 80)

print(f"\nConfiguration parameters:")
print(f"  Run 1: {len(runs_data['poxmea6n']['config'])} parameters")
print(f"  Run 2: {len(runs_data['buqt4b6u']['config'])} parameters")

print(f"\nSummary metrics:")
print(f"  Run 1: {len(runs_data['poxmea6n']['summary'])} metrics")
print(f"  Run 2: {len(runs_data['buqt4b6u']['summary'])} metrics")

print(f"\nCommon summary metrics: {len(common_summary_keys)}")
print(f"Common config keys: {len(common_config_keys)}")
print(f"Common history metrics: {len(common_metrics)}")

print(f"\n{'=' * 80}")
print("✓ Comparison complete!")
print("=" * 80)

---

## 📝 Notebook Guide

This notebook provides a comprehensive side-by-side comparison of two WandB experiment runs.

**What's included:**

| Section | Description |
|---------|-------------|
| **Imports** | Loads required libraries for data manipulation and visualization |
| **Run Definitions** | Defines the two run IDs: `poxmea6n` and `buqt4b6u` |
| **Data Loading** | Fetches metadata, config, metrics, history, files, and artifacts from both runs |
| **Comparison Tables** | Displays metadata, summary, and configuration side-by-side |
| **Training History** | Plots time-series metrics for both runs with different colors |
| **Performance Metrics** | Bar chart comparison of summary metrics |
| **Difference Analysis** | Calculates and visualizes percentage differences between runs |
| **Files/Artifacts** | Compares log files and artifacts between runs |
| **Images** | Downloads and displays all images from both runs side by side for visual comparison |
| **Summary** | Displays quick links and key statistics |

**To compare different runs:**
- Edit the `RUN_IDS` list in Cell 3

**Color coding:**
- 🔵 **Blue**: Run 1 (`poxmea6n`)
- 🟢 **Green**: Run 2 (`buqt4b6u`)
- ✅ **Green difference**: Run 2 performed better
- ❌ **Red difference**: Run 1 performed better